### Synopsis
This code averages and reduces g2 for temperature ramping (up and down) of protein H06

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle

import sys
sys.path.append('../common')
from utils import process_group

fpath1 ='/home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ'
fpath2 ='/home/8ididata/2022-1/babnigg202203/cluster_results_QZ'
coef_sam = 6.93e4  # Absolute scattering cross-section coefficient for the samples
coef_buf = 7.62e4  # Absolute scattering cross-section coefficient for the buffer


### Averaging of SA-XPCS results, temperature ramp up

In [2]:

avg_ramp_up, t_el, ql_dyn, ql_sta = process_group(group='E0110', 
                                              prefix=fpath2,
                                              zone_idx='auto',
                                              num_cores=24)


2023-12-25 15:44:53,953 utils                   : total number of files in E0110  is 1200
2023-12-25 15:44:54,632 utils                   : Failed to get temperature for file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0110_S3_4_Ramp_340C10p_att00_Rq0_00493_0001-100000.hdf, return nan
2023-12-25 15:44:54,828 utils                   : Failed to get temperature for file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0110_S3_4_Ramp_340C10p_att00_Rq0_00618_0001-100000.hdf, return nan
2023-12-25 15:44:56,046 xpcs_viewer.fileIO.hdf_reader: key not found: /xpcs/sphilist
2023-12-25 15:44:56,053 utils                   : failed to read file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0110_S3_4_Ramp_340C10p_att00_Rq0_00493_0001-100000.hdf, skip this file
2023-12-25 15:44:56,129 xpcs_viewer.fileIO.hdf_reader: key not found: /xpcs/sphilist
2023-12-25 15:44:56,136 utils                   : failed to read file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0

### Averaging of SA-XPCS results, temperature ramp up

In [3]:

avg_ramp_down, _, _, _ = process_group(group='E0111', 
                                 prefix=fpath2,
                                 zone_idx='auto',
                                 num_cores=24)


2023-12-25 15:44:59,748 utils                   : total number of files in E0111  is 1200
2023-12-25 15:45:01,669 xpcs_viewer.fileIO.hdf_reader: key not found: /xpcs/sphilist
2023-12-25 15:45:01,676 utils                   : failed to read file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0111_S3_4_Ramp_200C10p_att00_Rq0_00485_0001-100000.hdf, skip this file
2023-12-25 15:45:01,677 xpcs_viewer.fileIO.hdf_reader: key not found: /xpcs/sphilist
2023-12-25 15:45:01,683 utils                   : failed to read file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0111_S3_4_Ramp_200C10p_att00_Rq0_00006_0001-100000.hdf, skip this file
2023-12-25 15:45:02,426 xpcs_viewer.fileIO.hdf_reader: key not found: /xpcs/sphilist
2023-12-25 15:45:02,432 utils                   : failed to read file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0111_S3_4_Ramp_200C10p_att00_Rq0_00777_0001-100000.hdf, skip this file
2023-12-25 15:45:04,064 utils                   : label='E0111_s

### Averaging of background

In [4]:

avg_bg, _, _, _ = process_group(group='D0138',
                                 num_sections=1,
                                 prefix=fpath2,
                                 zone_idx='auto',
                                 num_cores=24)


2023-12-25 15:45:05,574 utils                   : total number of files in D0138  is 50
2023-12-25 15:45:06,916 utils                   : label='D0138_section_00': remove 7 datasets out of 50


### Data reduction

Subtracting I(Q) with background, converting to absolute scattering cross-section, and dump to pickle file for plotting

In [5]:

for ii in range(len(avg_ramp_up)):
    avg_ramp_up[ii]["saxs_1d"] = coef_sam*avg_ramp_up[ii]["saxs_1d"] - coef_buf*avg_bg[0]["saxs_1d"]

for ii in range(len(avg_ramp_down)):
    avg_ramp_down[ii]["saxs_1d"] = coef_sam*avg_ramp_down[ii]["saxs_1d"] - coef_buf*avg_bg[0]["saxs_1d"]

with open('checkpoint.pickle', 'wb') as f:  # 'wb' for binary write mode
    pickle.dump(avg_ramp_up, f)  # Dump the object to the file
    pickle.dump(avg_ramp_down, f)
    pickle.dump(t_el, f)
    pickle.dump(ql_sta, f)
    pickle.dump(ql_dyn, f)
    